<h1>IBM Mini Project</h1>
    <h1>Road Lane Line Detection</h1>

<h2>1) Importing necessary libraries</h2> 

In [1]:
import cv2 as cv
import numpy as np

In [2]:
#cv.__version__

<h2>2) Processing the video</h2>

In [3]:
# The input video has been read in as an object called VideoCapture
input_data = cv.VideoCapture("input.mp4")
while (input_data.isOpened()):
    # ret returns a boolean value from getting the frame and frame is taking the current frame being prossesed
    ret, frame = input_data.read()
    # Frames are processed in intervals of 10ms.'q' key is used to get out of the while loop
    cv.imshow('frame',frame)
    if cv.waitKey(10) & 0xFF == ord('q'):
        break

# The following frees up resources and closes all windows
input_data.release()
cv.destroyAllWindows()

<h2>3) Applying Canny Edge Detection </h2>

In [4]:
def do_canny(frame):
    # Converting the frame to grayscale as we only need to hilight the channel in edge detection 
    gray_scl = cv.cvtColor(frame, cv.COLOR_RGB2GRAY)
    #5x5 gaussian blur is applied with 0 to frame deviation
    blur = cv.GaussianBlur(gray_scl, (5, 5), 0)
    # Applying Canny edge detector with min val 50 and maxval 150
    canny = cv.Canny(blur, 50, 150)
    return canny

input_video = cv.VideoCapture("input.mp4")
while (input_video.isOpened()):
    ret, frame = input_video.read()

    canny = do_canny(frame)
    cv.imshow('canny',canny)
    if cv.waitKey(10) & 0xFF == ord('q'):
        break

input_video.release()
cv.destroyAllWindows()

<h2>4) Segmenting line area and masking other objects</h2>

In [5]:
def do_segment(frame):
    height = frame.shape[0]
    polygons = np.array([
                            [(0, height), (800, height), (380, 290)]
                        ])
    masking = np.zeros_like(frame)
    # the mask is given the value 1 and the other areas given 0
    cv.fillPoly(masking, polygons, 255)
    # bitwise and is implemented between mask and frame which retains the triangular area of frame
    segment = cv.bitwise_and(frame, masking)
    return segment
input_video = cv.VideoCapture("input.mp4")
while (input_video.isOpened()):
    ret, frame = input_video.read()
    seg = do_segment(frame)
    cv.imshow('segmenting',seg)
    if cv.waitKey(10) & 0xFF == ord('q'):
        break
input_video.release()
cv.destroyAllWindows()

<h2>5) Calculating lines</h2>

In [6]:
def calculate_lines(frame, lines):
    # Empty arrays store coordinates of both side of line
    left = []
    right = []
    for line in lines:
        #Changing line from 2D array to 1D array
        x1, y1, x2, y2 = line.reshape(4)
        #Fits a linear polynomial to the x and y coordinates and returns a vector of coefficients which describe the slope and y-intercept
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        slope = parameters[0]
        y_intercept = parameters[1]
        # If slope is negative, the line is to the left of the lane, and otherwise, the line is to the right of the lane
        if slope < 0:
            left.append((slope, y_intercept))
        else:
            right.append((slope, y_intercept))
    # Averages out all the values for left and right into a single slope and y-intercept value for each line
    left_avg = np.average(left, axis = 0)
    right_avg = np.average(right, axis = 0)
    # Calculates the x1, y1, x2, y2 coordinates for the left and right lines
    left_line = calculate_coordinates(frame, left_avg)
    right_line = calculate_coordinates(frame, right_avg)
    return np.array([left_line, right_line])

<h2>6) Calculating Coordinates</h2>

In [7]:
def calculate_coordinates(frame, parameters):
    slope, intercept = parameters
    # Sets initial y-coordinate as height from top down (bottom of the frame)
    y1 = frame.shape[0]
    # Sets final y-coordinate as 150 above the bottom of the frame
    y2 = int(y1 - 150)
    # Sets initial x-coordinate as (y1 - b) / m since y1 = mx1 + b
    x1 = int((y1 - intercept) / slope)
    # Sets final x-coordinate as (y2 - b) / m since y2 = mx2 + b
    x2 = int((y2 - intercept) / slope)
    return np.array([x1, y1, x2, y2])

<h2>7) For Visualizing the lines </h2>

In [8]:
def visualize_lines(frame, lines):
    # Creates an image filled with zero intensities with the same dimensions as the frame
    lines_visualize = np.zeros_like(frame)
    # Checks if any lines are detected
    if lines is not None:
        for x1, y1, x2, y2 in lines:
            # Draws lines between two coordinates with green color and 5 thickness
            cv.line(lines_visualize, (x1, y1), (x2, y2), (0, 255, 0), 5)
    return lines_visualize

<h2> 8) Output with lines being detected </h2> 

In [10]:
#The lines have been detected and looks like this
input_video = cv.VideoCapture("input.mp4")
while (input_video.isOpened()):
    ret, frame = input_video.read()
    segment = do_segment(canny)
    hough = cv.HoughLinesP(segment, 2, np.pi / 180, 100, np.array([]), minLineLength = 100, maxLineGap = 50)
    #Multiple detetcted lines are converted into a single line by hough transformation at both the corners of the lane
    lines = calculate_lines(frame, hough)
    # Visualising the lines
    lines_visualize = visualize_lines(frame, lines)
    cv.imshow("hough", lines_visualize)
    if cv.waitKey(10) & 0xFF == ord('q'):
        break
input_video.release()
cv.destroyAllWindows()

<h2> 10) Detecting the Vehicle </h2>

In [11]:
input_video = cv.VideoCapture('input.mp4')
car = cv.CascadeClassifier('cars.xml') 
while True: 
    ret, frames = input_video.read()  
    #convert to gray scale of each frames 
    gray = cv.cvtColor(frames, cv.COLOR_BGR2GRAY) 
    # Detects cars of different sizes in the input image 
    cars = car.detectMultiScale(gray, 1.1, 1) 
    # To draw a rectangle in each cars 
    for (x,y,w,h) in cars: 
        cv.rectangle(frames,(x,y),(x+w,y+h),(0,0,255),2)  
    cv.imshow('video', frames) 
    if cv.waitKey(10) & 0xFF == ord('q'):
        break
cv.destroyAllWindows() 

<h2>8) Visualizing the Final output</h2>

In [12]:
#the video with lines being detected has been implemented successfully
input_video = cv.VideoCapture("input.mp4")
car = cv.CascadeClassifier('cars.xml') 
while (input_video.isOpened()):
    ret, frame = input_video.read()
    canny = do_canny(frame)
   # cv.imshow("canny", canny)
    segment = do_segment(canny)
    hough = cv.HoughLinesP(segment, 2, np.pi / 180, 100, np.array([]), minLineLength = 100, maxLineGap = 50)
    # Averaging multiple detected lines from hough into one line for left border of lane and one line for right border of lane
    lines = calculate_lines(frame, hough)
    # Visualizing the lines
    lines_visualize = visualize_lines(frame, lines)
   # cv.imshow("hough", lines_visualize)
    # Overlaying lines on frame by taking their weighted sums and adding an arbitrary scalar value of 1 as the gamma argument
    ret, frames = input_video.read()  
    gray = cv.cvtColor(frames, cv.COLOR_BGR2GRAY) 
    cars = car.detectMultiScale(gray, 1.1, 1) 
    for (x,y,w,h) in cars: 
        cv.rectangle(frames,(x,y),(x+w,y+h),(0,0,255),2)  
    output = cv.addWeighted(frames, 0.9, lines_visualize, 1, 1)
    cv.imshow('output', output)
    if cv.waitKey(10) & 0xFF == ord('q'):
        break
input_video.release()
cv.destroyAllWindows()